<a href="https://colab.research.google.com/github/ProfAI/dl00/blob/master/9%20-%20Reti%20neurali%20ricorrenti%20(non%20rilasciata)/CNN_%2B_LSTM_con_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN + LSTM con Keras

In questo notebook proveremo a creare una rete neurale che combina uno strato convoluzionale con uno strato ricorrente per classificare le recensioni del IMDB Movie Reviews Dataset.
<br><br>
Importiamo i moduli che ci serviranno.

In [2]:
import numpy as np
import matplotlib.pyplot as plt

from keras.utils import to_categorical

from keras.models import Sequential
from keras.layers import Dense
from keras.datasets import imdb 
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


## Caricamento e preprocessing del dataset
Carichiamo il dataset con Keras limitandolo alle 10.000 parole più comuni, poi tronchiamo/espandiamo le sequenze a 500 elementi con la funzione pad_sequences.

In [3]:
num_words = 10000
maxlen = 500

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=num_words)

X_train = pad_sequences(X_train, maxlen = maxlen)
X_test = pad_sequences(X_test, maxlen = maxlen)

17465344/17464789 [==============================] - 0s 0us/step


## Creazione del modello

### Modello1: Rete convoluzionale

In [9]:
from keras.layers import Embedding, LSTM, Flatten
from keras.layers.convolutional import Conv1D, MaxPooling1D

model = Sequential()

model.add(Embedding(num_words, 50, input_length=500))
model.add(Conv1D(filters=32, kernel_size=3, padding="same", activation="relu"))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 500, 50)           500000    
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 500, 32)           4832      
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 250, 32)           0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 8000)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 8001      
Total params: 512,833
Trainable params: 512,833
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=512, validation_split=0.2, epochs=5)
model.evaluate(X_test, y_test)

Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 23s 1ms/step - loss: 0.6667 - acc: 0.6021 - val_loss: 0.6008 - val_acc: 0.6446
Epoch 2/5
20000/20000 [==============================] - 23s 1ms/step - loss: 0.4301 - acc: 0.8371 - val_loss: 0.4048 - val_acc: 0.8112
Epoch 3/5
20000/20000 [==============================] - 23s 1ms/step - loss: 0.2762 - acc: 0.8968 - val_loss: 0.3602 - val_acc: 0.8406
Epoch 4/5
20000/20000 [==============================] - 23s 1ms/step - loss: 0.2162 - acc: 0.9177 - val_loss: 0.2894 - val_acc: 0.8824
Epoch 5/5
25000/25000 [==============================] - 12s 470us/step


[0.3152529398679733, 0.87084]

## Modello 2: Da ricorrente a convoluzionale

In [0]:
from keras.layers import Embedding, LSTM, Flatten
from keras.layers.convolutional import Conv1D, MaxPooling1D

model = Sequential()

model.add(Embedding(num_words, 50, input_length=500))
model.add(LSTM(32, dropout=0.4, return_sequences=True))
model.add(Conv1D(filters=32, kernel_size=3, padding="same", activation="relu"))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 50)           500000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 500, 32)           10624     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 500, 32)           3104      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 250, 32)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 8000)              0         
_________________________________________________________________
dense_1 (Dense)      

Compiliamo il modello ed eseguiamo l'addestramento per 10 epoche.

In [0]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=512, validation_split=0.2, epochs=5)
model.evaluate(X_test, y_test)

Instructions for updating:
Use tf.cast instead.
Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 97s 5ms/step - loss: 0.6040 - acc: 0.6483 - val_loss: 0.5227 - val_acc: 0.7448
Epoch 2/5
20000/20000 [==============================] - 95s 5ms/step - loss: 0.3583 - acc: 0.8484 - val_loss: 0.4274 - val_acc: 0.8076
Epoch 3/5
20000/20000 [==============================] - 96s 5ms/step - loss: 0.2919 - acc: 0.8831 - val_loss: 0.3154 - val_acc: 0.8778
Epoch 4/5
20000/20000 [==============================] - 95s 5ms/step - loss: 0.2413 - acc: 0.9054 - val_loss: 0.2789 - val_acc: 0.8904
Epoch 5/5
25000/25000 [==============================] - 48s 2ms/step


[0.30027052153110506, 0.88164]

## Modello 3: Da convololuzionale a ricorrente
L'architettura della rete sarà la seguente:
1. Il primo strato esegue l'embedding creando 50 embedding vectors per le 10.000 parole nel nostro dizionario.
2. Il secondo strato è uno strato convoluzionale che usa 32 filtri di dimensione 3x3 per estrarre features dall'embedding.
3. Il terzo strato riduce la dimensione delle features map eseguendo il max pooling con una pool size di 2x2.
4. Il quarto strato è lo strato ricorrente (LSTM), per ridurre l'overfitting eseguiamo il dropout sull'input disattivando il 40% dei nodi.
5. Il quinto strato calcola l'output della rete.

In precedenza, per problemi di computer vision, abbiamo utilizzato la classe Conv2D di Keras per creare uno strato convoluzionale, questo strato prende in input un tensore (ti ricordo che un'immagine è rappresentata come un tensore le cui dimensioni sono altezza dell'immagine, larghezza dell'immagine e canali), questa volta il singolo esempio è una matrice composta dai word vectors delle varie parole che compono la frase, quindi dovremo utilizzare la classe Conv1D.

In [0]:
from keras.layers import Embedding, LSTM
from keras.layers.convolutional import Conv1D, MaxPooling1D

model = Sequential()

model.add(Embedding(num_words, 50))
model.add(Conv1D(filters=32, kernel_size=3, padding="same", activation="relu"))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(32, dropout=0.4))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 50)          500000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 32)          4832      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, None, 32)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 513,185
Trainable params: 513,185
Non-trainable params: 0
_______________

Compiliamo il modello ed eseguiamo l'addestramento per 10 epoche.

In [0]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=512, validation_split=0.2, epochs=5)
model.evaluate(X_test, y_test)

Instructions for updating:
Use tf.cast instead.
Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 54s 3ms/step - loss: 0.6425 - acc: 0.6269 - val_loss: 0.4521 - val_acc: 0.8152
Epoch 2/5
20000/20000 [==============================] - 53s 3ms/step - loss: 0.4325 - acc: 0.8062 - val_loss: 0.4439 - val_acc: 0.7962
Epoch 3/5
20000/20000 [==============================] - 53s 3ms/step - loss: 0.3350 - acc: 0.8593 - val_loss: 0.3137 - val_acc: 0.8748
Epoch 4/5
20000/20000 [==============================] - 53s 3ms/step - loss: 0.2806 - acc: 0.8859 - val_loss: 0.2885 - val_acc: 0.8822
Epoch 5/5
25000/25000 [==============================] - 24s 980us/step


[0.2801623426914215, 0.88612]

Come vedi, questo approccio ibrido in cui abbiamo combinato uno strato convoluzionale con uno strato ricorrente ci ha portato al miglior risultato che abbiamo ottenuto fin'ora.